# Simulating lightcurves of lensed SNe Ia

This example requires the development version of simurvey found in this feature branch:
https://github.com/ufeindt/simsurvey/tree/feature/manipulate-transient-parameters

The changes there allow easy manipulation of the transient parameters, e.g. to change the amplitude based on lens magnification or duplicate a transient and shift its RA, Dec and $t_0$ slightly to simulate multiple images.

In [1]:
import os
home_dir = os.environ.get('HOME')

# Please enter the path to where you have placed the Schlegel, Finkbeiner & Davis (1998) dust map files
# You can also set the environment variable SFD_DIR to this path (in that case the variable below should be None)
sfd98_dir = os.path.join(home_dir, 'data/sfd98')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.cosmology import Planck15
import simsurvey
import sncosmo
import simsurvey_tools as sst

In [3]:
simsurvey.__version__

'0.7.0.dev3'

In [4]:
simsurvey.__path__

['/home/ufeindt/anaconda3/envs/simsurvey-dev/lib/python3.7/site-packages/simsurvey-0.7.0.dev3-py3.7.egg/simsurvey']

In [5]:
fields = sst.load_ztf_fields()

In [6]:
fields['ra'][682], fields['dec'][682]

(274.7095, 33.35)

In [7]:
# Load the CCD corners from file
ccd_corners = np.genfromtxt('data/ztf_quadrants.tbl', skip_header=1)
ccds = [ccd_corners[np.array([0,1,2,3])+4*k, :2] for k in range(64)]

In [8]:
bands = { 
  'ztfr' : 'data/ztfr_eff.txt',
  'ztfg' : 'data/ztfg_eff.txt',
  'ztfi' : 'data/ztfi_eff.txt',
  }

for bandname in bands.keys() :
    fname = bands[bandname]
    b = np.loadtxt(fname)
    band = sncosmo.Bandpass(b[:,0], b[:,1], name=bandname)
    sncosmo.registry.register(band, force=True)

In [9]:
obs = {'time': [], 'field': [], 'band': [], 'skynoise': [], 'comment': [], 'zp': []}

mjd_start = 58239.5
for k in range(0, 61, 3):
    obs['time'].extend([mjd_start + k + l/24. for l in range(2)])
    obs['field'].extend([683 for l in range(2)])
    obs['band'].extend(['ztfg', 'ztfr'])
    obs['skynoise'].extend([1262 for l in range(2)])
    obs['zp'].extend([30 for l in range(2)])
    obs['comment'].extend(['MSIP', 'MSIP'])
    
#for k in range(61):
#    obs['time'].extend([mjd_start + k + (l+2)/24. for l in range(3)])
#    obs['field'].extend([686 for l in range(3)])
#    obs['band'].extend(['ztfg', 'ztfg', 'ztfg'])
#    obs['skynoise'].extend([1262 for l in range(3)])
#    obs['zp'].extend([30 for l in range(3)])
#    obs['comment'].extend(['fast-g', 'fast-g', 'fast-g'])    
     
plan = simsurvey.SurveyPlan(time=obs['time'], band=obs['band'], obs_field=obs['field'],
                            skynoise=obs['skynoise'], comment=obs['comment'],
                            zp=obs['zp'],
                            fields={k: v for k, v in fields.items()
                                    if k in ['ra', 'dec', 'field_id',
                                            'width', 'height']},
                            ccds=ccds)

In [10]:
plan.pointings

time,band,zp,skynoise,RA,Dec,field,ccd,comment
float64,str4,int64,int64,float64,float64,int64,float64,str4
58239.5,ztfg,30,1262,274.7095,33.35,683,nan,MSIP
58239.541666666664,ztfr,30,1262,274.7095,33.35,683,nan,MSIP
58242.5,ztfg,30,1262,274.7095,33.35,683,nan,MSIP
58242.541666666664,ztfr,30,1262,274.7095,33.35,683,nan,MSIP
58245.5,ztfg,30,1262,274.7095,33.35,683,nan,MSIP
58245.541666666664,ztfr,30,1262,274.7095,33.35,683,nan,MSIP
58248.5,ztfg,30,1262,274.7095,33.35,683,nan,MSIP
58248.541666666664,ztfr,30,1262,274.7095,33.35,683,nan,MSIP
58251.5,ztfg,30,1262,274.7095,33.35,683,nan,MSIP


In [11]:
mjd_range = (plan.pointings['time'].min(), plan.pointings['time'].max())
ra_range = (270., 280.)
dec_range = (29., 37.5)

In [12]:
z_max = 0.1

tr = simsurvey.get_transient_generator([0., z_max],
                                       transient='Ia', 
                                       template='salt2',
                                       ra_range=ra_range,
                                       dec_range=dec_range,
                                       mjd_range=[mjd_range[0],
                                                  mjd_range[1]],
                                       sfd98_dir=sfd98_dir,
                                       ntransient=6)

## New functions
### get_lc_param()
This function allows quick retrieval of lightcurve parameters. You can pass a single index, a numpy.array of integers, or a numpy.array of boolean (mask) to select the transients.

In [13]:
print("Redshifts of all SNe (for reference):", tr.zcmb)
print()
print("Retrieve 3rd SNe - tr.get_lc_param(2)")
param = tr.get_lc_param(2)
print(param)

Redshifts of all SNe (for reference): [0.09396139 0.09869869 0.09148486 0.09176402 0.06753981 0.07403427]

Retrieve 3rd SNe - tr.get_lc_param(2)
{'z': 0.09148485917726448, 't0': 58294.78291490701, 'ra': 276.83266247981857, 'dec': 32.93141426711464, 'x0': 0.00032912509106388987, 'x1': -0.3406508437401799, 'c': 0.12883355928841075, 'mwebv': 0.08281060570135053, 'mwebv_sfd98': 0.09184049489803311}


In [14]:
print("Retrieve 1st and 3rd SNe using a integer array")
param_mult = tr.get_lc_param(np.array([0, 2]))
print(param_mult)

Retrieve 1st and 3rd SNe using a integer array
{'z': array([0.09396139, 0.09148486]), 't0': array([58249.32693146, 58294.78291491]), 'ra': array([277.54918454, 276.83266248]), 'dec': array([34.05255239, 32.93141427]), 'x0': array([0.00039244, 0.00032913]), 'x1': array([-0.68140078, -0.34065084]), 'c': array([0.0238902 , 0.12883356]), 'mwebv': array([0.06776928, 0.08281061]), 'mwebv_sfd98': array([0.05792018, 0.09184049])}


In [15]:
print("Retrieve 1st and 3rd SNe using a boolean array")
mask = np.zeros(tr.ntransient, dtype=bool)
mask[0] = True
mask[2] = True
param_mult = tr.get_lc_param(mask)
print(param_mult)

Retrieve 1st and 3rd SNe using a boolean array
{'z': array([0.09396139, 0.09148486]), 't0': array([58249.32693146, 58294.78291491]), 'ra': array([277.54918454, 276.83266248]), 'dec': array([34.05255239, 32.93141427]), 'x0': array([0.00039244, 0.00032913]), 'x1': array([-0.68140078, -0.34065084]), 'c': array([0.0238902 , 0.12883356]), 'mwebv': array([0.06776928, 0.08281061]), 'mwebv_sfd98': array([0.05792018, 0.09184049])}


### remove_lc_param()
This function removes a transient from the generator. Again multiple

In [16]:
print("Redshifts of all SNe (before):", tr.zcmb)
tr.remove_lc_param(2)
print("Redshifts of all SNe (after removing the 3rd SN):", tr.zcmb)

Redshifts of all SNe (before): [0.09396139 0.09869869 0.09148486 0.09176402 0.06753981 0.07403427]
Redshifts of all SNe (after removing the 3rd SN): [0.09396139 0.09869869 0.09176402 0.06753981 0.07403427]


In [17]:
print("Redshifts of all SNe (before):", tr.zcmb)
tr.remove_lc_param(np.array([1, 3]))
print("Redshifts of all SNe (after removing the 2nd and 4th SN):", tr.zcmb)

Redshifts of all SNe (before): [0.09396139 0.09869869 0.09176402 0.06753981 0.07403427]
Redshifts of all SNe (after removing the 2nd and 4th SN): [0.09396139 0.09176402 0.07403427]


### add_lc_param()
This function adds a new SN. The easiest way to generate the correct input is to get a dictionary of parameters and change it.

In [18]:
print("Redshifts of all SNe:", tr.zcmb)
tr.add_lc_param(**param)
print("Redshifts of all SNe (after adding a SN):", tr.zcmb)

Redshifts of all SNe: [0.09396139 0.09176402 0.07403427]
Redshifts of all SNe (after adding a SN): [0.09396139 0.09176402 0.07403427 0.09148486]


In [19]:
print("Redshifts of all SNe:", tr.zcmb)
tr.add_lc_param(**param_mult)
print("Redshifts of all SNe (after removing the 3rd SN):", tr.zcmb)

Redshifts of all SNe: [0.09396139 0.09176402 0.07403427 0.09148486]
Redshifts of all SNe (after removing the 3rd SN): [0.09396139 0.09176402 0.07403427 0.09148486 0.09396139 0.09148486]


### pop_lc_param()
This function combines `get_lc_param()` and `remove_lc_param()` into one step.

In [20]:
print("Redshifts of all SNe (before):", tr.zcmb)
print()
print("Pop 3rd SNe - tr.pop_lc_param(2)")
param = tr.pop_lc_param(2)
print(param)
print()
print("Redshifts of all SNe (after popping the 3rd SN):", tr.zcmb)

Redshifts of all SNe (before): [0.09396139 0.09176402 0.07403427 0.09148486 0.09396139 0.09148486]

Pop 3rd SNe - tr.pop_lc_param(2)
{'z': 0.07403426596691835, 't0': 58240.40509261257, 'ra': 272.012624644036, 'dec': 31.880318863940822, 'x0': 0.0006407998241353049, 'x1': -1.3239677652114972, 'c': -0.03293368115945299, 'mwebv': 0.051591177497025374, 'mwebv_sfd98': 0.05116327440625089}

Redshifts of all SNe (after popping the 3rd SN): [0.09396139 0.09176402 0.09148486 0.09396139 0.09148486]


### set_lc_param()
If you just wish to change a few parameters of a transient, use this function.

In [21]:
print('3rd SN:')
param = tr.get_lc_param(2)
print(param)
print()
tr.set_lc_param(2, t0=param['t0']+10, x0=param['x0']*10)
print('3rd SN with magnification and time delay:')
param = tr.get_lc_param(2)
print(param)

3rd SN:
{'z': 0.09148485917726448, 't0': 58294.78291490701, 'ra': 276.83266247981857, 'dec': 32.93141426711464, 'x0': 0.00032912509106388987, 'x1': -0.3406508437401799, 'c': 0.12883355928841075, 'mwebv': 0.08281060570135053, 'mwebv_sfd98': 0.09184049489803311}

3rd SN with magnification and time delay:
{'z': 0.09148485917726448, 't0': 58304.78291490701, 'ra': 276.83266247981857, 'dec': 32.93141426711464, 'x0': 0.003291250910638899, 'x1': -0.3406508437401799, 'c': 0.12883355928841075, 'mwebv': 0.08281060570135053, 'mwebv_sfd98': 0.09184049489803311}


## The simulation should hopefully still run properly

In [22]:
survey = simsurvey.SimulSurvey(generator=tr, plan=plan)

lcs = survey.get_lightcurves(
    progress_bar=True, notebook=True # If you get an error because of the progress_bar, delete this line.
)
print(tr.ntransient, len(lcs.lcs))

Determining field IDs for all objects


FloatProgress(value=0.0)


Generating lightcurves


FloatProgress(value=0.0)


5 5
